## 1.Imports and Configuration

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time 
import os
from urllib.parse import quote_plus
import yfinance as yf

# Database Interaction
import mysql.connector # Import the driver
from sqlalchemy import create_engine, text # For pandas integration


# import credentials from config file (if using)
try:
    from config import ALPHA_VANTAGE_API_KEY, MYSQL_HOST, MYSQL_USER, MYSQL_PASSWORD, MYSQL_DATABASE
except ImportError:
    print("Error: config.py not found or variables missing.")
    print("Please create config.py with your API Key and MySQL credentials.")
    # You could prompt the user here or set default placeholder values
    APLHA_VANTAGE_API_KEY = 'YOUR_API_KEY_PLACEHOLDER'
    MYSQL_HOST = 'localhost'
    MYSQL_USER = 'root'
    MYSQL_PASSWORD = 'password'
    MYSQL_DATABASE = 'stock_market_db'


# --- Project Configuration ---
TICKERS = ['AAPL','MSFT','GOOGL'] # Stocks to fetch
API_URL = 'https://www.alphavantage.co/query'
FUNCTION = 'TIME_SERIES_DAILY_ADJUSTED'
OUTPUT_SIZE = 'compact'  # 'compact' for 100 days, 'full' for more history. We're starting with compact for testing


# Moving Average Windows
MA_SHORT = 20 # Shorter window for compact data
MA_LONG = 50 # Longer window


# Output Configuration
OUTPUT_DIR = 'output_mysql' # Directory to save plots and Excel file
EXCEL_FILENAME = os.path.join(OUTPUT_DIR, 'stock_analyses_report_mysql.xlsx')
PLOT_FILENAME_TEMPLATE = os.path.join(OUTPUT_DIR, '{}_price_MA_plot_mysql.png')

# Ensure output directory exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Created output directory: {OUTPUT_DIR}")

# For inline plots in Jupyter
%matplotlib inline

## 2.Database Connection Setup

In [3]:
# --- Cell 2 Cleaned ---
try:
    # URL-encode the password if it contains special characters
    encoded_password = quote_plus(MYSQL_PASSWORD)

    # Build the connection string using the encoded password
    connection_string = f"mysql+mysqlconnector://{MYSQL_USER}:{encoded_password}@{MYSQL_HOST}/{MYSQL_DATABASE}"

    engine = create_engine(connection_string)

    # Test connection
    with engine.connect() as connection:
        print("Successfully connected to MySQL database!") # KEEP

except NameError as ne: # Keep error handling
    print(f"Caught NameError: {ne}. Did you forget to run Cell 1 after restarting the kernel?")
    engine=None
except ImportError: # Keep error handling
    print("Error: mysql-connector-python or sqlalchemy not installed properly.")
    engine=None
except Exception as e: # Keep error handling
    print(f"Error connecting to MySQL database: {e}")
    print("Please check your MySQL server status and credentials in config.py.")
    engine = None

Successfully connected to MySQL database!


## 3.Create Database Table (if it doesn't exist)

In [5]:
# --- Create Table Definition ---
# Note: You can also create this table directly in MySQL Workbench first.
# Using DECIMAL for prices is generally better for financial data than FLOAT/REAL.
# VARCHAR is suitable for the symbol and date string.
# Define the table name
TABLE_NAME = 'daily_stock_data'

create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
    symbol VARCHAR(10) NOT NULL,
    date DATE NOT NULL,        -- Use DATE type for dates in MySQL
    open DECIMAL(10, 4),       -- Precision 10, 4 decimal places
    high DECIMAL(10, 4),
    low DECIMAL(10, 4),
    close DECIMAL(10, 4),
    adjusted_close DECIMAL(10, 4),
    volume BIGINT,             -- Use BIGINT for potentially large volumes
    PRIMARY KEY (symbol, date) -- Composite primary key prevents duplicates
);
"""

# Execute the CREATE TABLE statement
if engine:
    try:
        with engine.connect() as connection:
            connection.execute(text(create_table_sql))
            connection.commit() 
        print(f"Table '{TABLE_NAME}' created successfully or already exists.")
    except Exception as e:
        print(f"Error creating table '{TABLE_NAME}': {e}")
else:
    print("Skipping table creation due to connection error.")

Table 'daily_stock_data' created successfully or already exists.


## 4: Data Fetching Function

In [8]:
# --- Cell 4 Cleaned ---
import yfinance as yf # Make sure imports are here or in Cell 1
import pandas as pd

def fetch_stock_data_yf(symbol):
    print(f"Fetching data for {symbol} from Yahoo Finance...") # KEEP
    try:
        ticker = yf.Ticker(symbol)
        df = ticker.history(period="5y")

        if df.empty:
            print(f"No data returned for {symbol} from yfinance.") # KEEP
            return None

        # --- Column renaming logic as developed ---
        df.rename(columns={
            'Open': 'open', 'High': 'high', 'Low': 'low',
            'Close': 'close', 'Volume': 'volume'
        }, inplace=True)
        if 'Adj Close' in df.columns:
             df.rename(columns={'Adj Close': 'adjusted_close'}, inplace=True)
             cols_to_keep = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume']
        else:
             df['adjusted_close'] = df['close']
             cols_to_keep = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume']
        df = df[cols_to_keep]
        # --- End renaming logic ---

        df['symbol'] = symbol
        df.reset_index(inplace=True)
        df.rename(columns={'Date': 'date'}, inplace=True)
        df['date'] = pd.to_datetime(df['date']).dt.date

        print(f"Successfully fetched {len(df)} data points for {symbol} using yfinance") # KEEP
        return df[['symbol', 'date', 'open', 'high', 'low', 'close', 'adjusted_close', 'volume']]

    except Exception as e:
        print(f"An error occurred fetching {symbol} using yfinance: {e}") # KEEP
        return None

## 5: Data Storage Function

In [13]:
def save_to_mysql(df, table_name, db_engine):
    """Saves the DataFrame to the MySQL database, handling duplicates via INSERT IGNORE."""
    if df is None or df.empty or db_engine is None:
        print("No data/engine available to save.")
        return 0 # Return count of rows added

    symbol = df['symbol'].iloc[0] if not df.empty else 'Unknown'
    rows_added = 0

    # Use pandas `to_sql` with a custom method for handling duplicates ('INSERT IGNORE')
    # A more robust way than relying solely on `if_exists='append'` with primary key errors.
    # Need to write raw SQL or use a temp table approach for complex upserts.
    # Simplest here: try appending and let the primary key handle it (will log errors).
    # OR: fetch existing dates and filter df before inserting. Let's try the PK approach first.

    try:
        # 'append' will try to insert all rows. The PRIMARY KEY constraint
        # in MySQL will cause errors for duplicates, which pandas might raise.
        df.to_sql(name=table_name, con=db_engine, if_exists='append', index=False)
        # Note: `to_sql` with `append` doesn't return the number of rows actually inserted
        # if some were duplicates. We assume success if no exception is raised.
        # A more accurate count would require checking before/after counts or complex SQL.
        print(f"Attempted to save/append data for {symbol} to table '{table_name}'. Check MySQL logs for duplicate key errors if any.")
        # Simplistic row count - assumes all were potentially new
        rows_added = len(df)

    # Catch the specific MySQL IntegrityError if possible (might be wrapped by SQLAlchemy)
    except Exception as e: # Catching a broader exception for now
        if 'Duplicate entry' in str(e):
             print(f"Duplicate key error encountered for {symbol}. Some rows may not have been inserted.")
             # You could implement logic here to filter out existing dates and retry insertion.
        else:
             print(f"Database error saving {symbol}: {e}")
    return rows_added # Return the attempted number of rows

##  6: Main Fetching and Storing Loop

In [16]:
# --- Fetch and Store Data for All Tickers ---
total_rows_added = 0
if engine: # Only proceed if the database connection is okay
    for ticker in TICKERS:
        print(f"\n--- Processing {ticker} ---")
        stock_df = fetch_stock_data_yf(ticker)

        if stock_df is not None:
            rows = save_to_mysql(stock_df, TABLE_NAME, engine)
            total_rows_added += rows
            # You can verify insertion by querying the table in MySQL Workbench
            # SELECT * FROM daily_stock_data WHERE symbol = 'AAPL' ORDER BY date DESC LIMIT 5;
        else:
             print(f"Skipping database save for {ticker} due to fetch error.")

        # IMPORTANT: Respect Alpha Vantage free tier limits (e.g., 5 calls per minute)
        print("Pausing for 15 seconds...")
        time.sleep(15)

    print(f"\nData fetching and storing process completed. Attempted to add {total_rows_added} rows (duplicates ignored by DB).")
else:
    print("Database connection failed. Cannot fetch or store data.")


--- Processing AAPL ---
Fetching data for AAPL from Yahoo Finance...
Successfully fetched 1257 data points for AAPL using yfinance
Duplicate key error encountered for AAPL. Some rows may not have been inserted.
Pausing for 15 seconds...

--- Processing MSFT ---
Fetching data for MSFT from Yahoo Finance...
Successfully fetched 1257 data points for MSFT using yfinance
Duplicate key error encountered for MSFT. Some rows may not have been inserted.
Pausing for 15 seconds...

--- Processing GOOGL ---
Fetching data for GOOGL from Yahoo Finance...
Successfully fetched 1257 data points for GOOGL using yfinance
Duplicate key error encountered for GOOGL. Some rows may not have been inserted.
Pausing for 15 seconds...

Data fetching and storing process completed. Attempted to add 0 rows (duplicates ignored by DB).


## 7: Data Loading Function (from MySQL))


def load_data_from_mysql(symbol, table_name, db_engine):
    """Loads stock data for a specific symbol from the MySQL database."""
    if db_engine is None:
        print("Database engine not available.")
        return None

    print(f"\nLoading data for {symbol} from MySQL...")
    try:
        # Use pandas read_sql_query for safety and convenience
        query = f"SELECT * FROM {table_name} WHERE symbol = %s ORDER BY date ASC"
        # Use %s placeholder for mysqlconnector, SQLAlchemy handles parameterization
        df = pd.read_sql_query(query, db_engine, params=(symbol,))

        if df.empty:
            print(f"No data found in database for symbol: {symbol}")
            return None

        # Convert 'date' column to datetime objects (if not already) and set as index
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)

        # Convert decimal columns back to float for analysis if needed (or keep as object/Decimal)
        # Pandas usually reads them as 'object' type representing Decimal. Convert to float for plotting/calculations.
        for col in ['open', 'high', 'low', 'close', 'adjusted_close']:
             if col in df.columns:
                 df[col] = pd.to_numeric(df[col]) # Handles Decimal conversion

        df['volume'] = pd.to_numeric(df['volume'])

        print(f"Loaded {len(df)} data points for {symbol} from database.")
        return df

    except Exception as e:
        print(f"Error loading data for {symbol} from MySQL: {e}")
        return None

## 8: Analysis Function (Moving Averages)

In [20]:
def calculate_moving_averages_pd(df, short_window=MA_SHORT, long_window=MA_LONG):
    """Calculates short-term and long-term moving averages using Pandas."""
    if df is None or 'adjusted_close' not in df.columns:
        print("DataFrame is None or missing 'adjusted_close' column.")
        return None

    # Ensure the column is numeric
    df['adjusted_close'] = pd.to_numeric(df['adjusted_close'])

    df[f'MA_{short_window}'] = df['adjusted_close'].rolling(window=short_window).mean()
    df[f'MA_{long_window}'] = df['adjusted_close'].rolling(window=long_window).mean()
    print(f"Calculated {short_window}-day and {long_window}-day Moving Averages.")
    return df

## 9: Plotting Function)

In [22]:
def plot_stock_data(df, symbol):
    """Generates and saves a plot of adjusted close price and moving averages."""
    if df is None:
        print(f"Cannot plot data for {symbol}, DataFrame is None.")
        return

    plt.figure(figsize=(12, 6)) # Adjusted size slightly for notebook view
    plt.plot(df.index, df['adjusted_close'], label='Adjusted Close', alpha=0.9)

    ma_short_col = f'MA_{MA_SHORT}'
    ma_long_col = f'MA_{MA_LONG}'

    if ma_short_col in df.columns and not df[ma_short_col].isnull().all():
      plt.plot(df.index, df[ma_short_col], label=f'{MA_SHORT}-Day MA', linestyle='--')
    if ma_long_col in df.columns and not df[ma_long_col].isnull().all():
      plt.plot(df.index, df[ma_long_col], label=f'{MA_LONG}-Day MA', linestyle=':')

    plt.title(f'{symbol} - Adjusted Close Price and Moving Averages (MySQL Data)')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout() # Adjust layout

    # Save the plot
    plot_filename = PLOT_FILENAME_TEMPLATE.format(symbol)
    try:
        plt.savefig(plot_filename)
        print(f"Plot saved to {plot_filename}")
    except Exception as e:
        print(f"Error saving plot {plot_filename}: {e}")

    plt.show() # Display the plot inline in Jupyter

## 10: Excel Export Function

In [24]:
def export_analysis_to_excel(data_dict, filename):
    """Exports the analyzed dataframes to separate sheets in an Excel file."""
    if not data_dict:
        print("No data to export.")
        return

    print(f"\nExporting analysis to Excel: {filename}")
    try:
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            for symbol, df in data_dict.items():
                if df is not None:
                    # Select relevant columns, reset index to include date
                    cols_to_export = ['adjusted_close', f'MA_{MA_SHORT}', f'MA_{MA_LONG}']
                    df_to_export = df[cols_to_export].copy()
                    df_to_export.reset_index(inplace=True) # Date becomes a column
                    # Format date for Excel readability
                    df_to_export['date'] = df_to_export['date'].dt.strftime('%Y-%m-%d')

                    # Handle potential missing MA columns if windows were too large for data
                    for col in cols_to_export:
                        if col not in df_to_export.columns:
                            df_to_export[col] = None # Add empty column if missing

                    df_to_export.to_excel(writer, sheet_name=symbol, index=False)
            print(f"Analysis results exported successfully to {filename}")
    except Exception as e:
        print(f"Error exporting data to Excel: {e}")

## 11: Main Analysis Loop

In [26]:
# --- Run the Analysis and Reporting ---
analyzed_data_mysql = {} # Dictionary to hold dataframes for Excel export

if engine: # Only proceed if database connection is okay
    for ticker in TICKERS:
        # 1. Load data using SQL (via Python/SQLAlchemy)
        stock_df = load_data_from_mysql(ticker, TABLE_NAME, engine)

        if stock_df is not None:
            # 2. Perform Analysis (Python/Pandas)
            stock_df = calculate_moving_averages_pd(stock_df)

            # 3. Visualize (Python/Matplotlib - displayed inline)
            plot_stock_data(stock_df, ticker)

            # Store for Excel export
            analyzed_data_mysql[ticker] = stock_df
        else:
            print(f"Skipping analysis and plotting for {ticker} due to loading error.")

    # 4. Export to Excel (Python/Pandas)
    if analyzed_data_mysql:
        export_analysis_to_excel(analyzed_data_mysql, EXCEL_FILENAME)
    else:
        print("No data was analyzed, skipping Excel export.")

    print("\nAnalysis and reporting process completed.")

else:
    print("Database connection failed. Cannot run analysis.")

# Optional: Close the engine when completely done (though usually not necessary for scripts)
# if engine:
#    engine.dispose()

NameError: name 'load_data_from_mysql' is not defined

## 12: Using MySQL Workbench

### Using MySQL Workbench

Now that the script has run:

1.  **Open MySQL Workbench** and connect to your server.
2.  **Navigate** to the `stock_market_db` schema (database).
3.  **Expand Tables:** You should see the `daily_stock_data` table.
4.  **Inspect Data:** Right-click on `daily_stock_data` and choose "Select Rows - Limit 1000". You'll see the raw data stored by the Python script.
5.  **Run Queries:** You can run your own SQL queries directly in Workbench to explore the data further, for example:
    ```sql
    -- Get the latest entry for each stock
    SELECT symbol, MAX(date) AS latest_date
    FROM daily_stock_data
    GROUP BY symbol;

    -- Count rows per stock
    SELECT symbol, COUNT(*) AS row_count
    FROM daily_stock_data
    GROUP BY symbol;

    -- Find days with large price swings for AAPL (e.g., high-low > 5% of open)
    SELECT date, open, high, low, close
    FROM daily_stock_data
    WHERE symbol = 'AAPL' AND (high - low) > (open * 0.05)
    ORDER BY date DESC;
    ```
This demonstrates your ability to use SQL *independently* to query the database created and populated by your Python process.

### Interpretation and Next Steps

*   **Moving Averages:** The plots show the stock's adjusted closing price along with its short-term (e.g., 20-day) and long-term (e.g., 50-day) moving averages.
    *   When the price is above the MAs, it can indicate an uptrend.
    *   When the short-term MA crosses above the long-term MA (a "golden cross"), it's often seen as a bullish signal.
    *   When the short-term MA crosses below the long-term MA (a "death cross"), it's often seen as a bearish signal.
*   **Excel Report:** The generated `stock_analysis_report_mysql.xlsx` file contains the raw adjusted close and calculated MAs for each stock on separate sheets. This file could be shared or used for further ad-hoc analysis or dashboarding in Excel itself.
*   **Potential Improvements:**
    *   Add more technical indicators (RSI, MACD).
    *   Calculate daily/weekly returns and volatility.
    *   Incorporate volume analysis alongside price.
    *   Implement error handling for API rate limits more gracefully.
    *   Build an interactive dashboard (e.g., using Streamlit or Dash) instead of static plots/Excel.
    *   Optimize database insertion for very large datasets (e.g., bulk inserts, handling duplicates more efficiently).